In [1]:
import acquire as a
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

## First thing I want to do is standardize all the column names. 

In [2]:
df = a.acquire_edu_data()

In [3]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Gender', 'EthnicGroup', 'ParentEduc',
       'LunchType', 'TestPrep', 'ParentMaritalStatus', 'PracticeSport',
       'IsFirstChild', 'NrSiblings', 'TransportMeans', 'WklyStudyHours',
       'MathScore', 'ReadingScore', 'WritingScore'],
      dtype='object')

    Key takaways:
    - there is a combination of upper and lowercase letters
    - Some columns have sapces 
    - unnamed columns are mirror of index so we can remove those columns. 
    

In [4]:
# lets addess the first two with one line of code
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [5]:
# now for the unnamed columns
df.drop(columns=[c for c in df.columns if 'unnamed' in c],inplace=True)


## Here we are looking to address the null values . This is a special case with nulls becasue every data point is a student. To use df.dropna() will drop a student who didnt have a voice and we don't want to take that away from a student.

In [6]:
# first I want to see how many missing values there are in the dataset and set it to output it by % of columns
(df.isna().sum() / len(df) )* 100 

gender                  0.000000
ethnicgroup             6.005026
parenteduc              6.021344
lunchtype               0.000000
testprep                5.972390
parentmaritalstatus     3.883685
practicesport           2.059332
isfirstchild            2.950295
nrsiblings              5.130381
transportmeans         10.228126
wklystudyhours          3.116739
mathscore               0.000000
readingscore            0.000000
writingscore            0.000000
dtype: float64

    Addressing the first null, I'm gonna drop the column ethnicgroup. I did this to prevent any potential biases or unfair labeling based on ethnicity, and to ensure that the analysis is focused solely on the other factors that may be affecting educational performance. It is important to note that removing a variable like ethnicity from the analysis does not mean that it is not an important factor, but rather that in this specific analysis, we are choosing to focus on other variables.

In [7]:
df.drop(columns='ethnicgroup', inplace=True)

In [8]:
# first I want to see how many missing values there are in the dataset and set it to output it by % of columns
(df.isna().sum() / len(df) )* 100 

gender                  0.000000
parenteduc              6.021344
lunchtype               0.000000
testprep                5.972390
parentmaritalstatus     3.883685
practicesport           2.059332
isfirstchild            2.950295
nrsiblings              5.130381
transportmeans         10.228126
wklystudyhours          3.116739
mathscore               0.000000
readingscore            0.000000
writingscore            0.000000
dtype: float64

    We are going to impute the nulls until they are able to be update. Since all the nulls are less than 11 % of the columns in the dataset I feel comfortable enough to impute them.

In [9]:
has_null =[]

for cols in df.columns:
    if df[cols].isna().sum() > 0:
        has_null.append(cols)
    else:
        pass

In [10]:
# Here we trust but verify our code, and ensure that the null columns were added to the list
has_null

['parenteduc',
 'testprep',
 'parentmaritalstatus',
 'practicesport',
 'isfirstchild',
 'nrsiblings',
 'transportmeans',
 'wklystudyhours']

In [11]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')

# Fill the missing values in each column
for col in df.columns:
    if df[col].isna().sum() > 0:
        df[col] = imputer.fit_transform(df[col].values.reshape(-1, 1))


In [12]:
(df.isna().sum() / len(df) )* 100 

gender                 0.0
parenteduc             0.0
lunchtype              0.0
testprep               0.0
parentmaritalstatus    0.0
practicesport          0.0
isfirstchild           0.0
nrsiblings             0.0
transportmeans         0.0
wklystudyhours         0.0
mathscore              0.0
readingscore           0.0
writingscore           0.0
dtype: float64

In [13]:
df.testprep.value_counts()

none         20686
completed     9955
Name: testprep, dtype: int64

In [14]:
df.describe()

,nrsiblings,mathscore,readingscore,writingscore
count,30641.000000,30641.000000,30641.000000,30641.000000
mean,2.087106,66.558402,69.377533,68.418622
std,1.442665,15.361616,14.758952,15.443525
min,0.000000,0.000000,10.000000,4.000000
25%,1.000000,56.000000,59.000000,58.000000
50%,2.000000,67.000000,70.000000,69.000000
75%,3.000000,78.000000,80.000000,79.000000
max,7.000000,100.000000,100.000000,100.000000


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30641 entries, 0 to 30640
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               30641 non-null  object 
 1   parenteduc           30641 non-null  object 
 2   lunchtype            30641 non-null  object 
 3   testprep             30641 non-null  object 
 4   parentmaritalstatus  30641 non-null  object 
 5   practicesport        30641 non-null  object 
 6   isfirstchild         30641 non-null  object 
 7   nrsiblings           30641 non-null  float64
 8   transportmeans       30641 non-null  object 
 9   wklystudyhours       30641 non-null  object 
 10  mathscore            30641 non-null  int64  
 11  readingscore         30641 non-null  int64  
 12  writingscore         30641 non-null  int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 3.0+ MB


In [16]:
df.head()

,gender,parenteduc,lunchtype,testprep,parentmaritalstatus,practicesport,isfirstchild,nrsiblings,transportmeans,wklystudyhours,mathscore,readingscore,writingscore
0,female,bachelor's degree,standard,none,married,regularly,yes,3.0,school_bus,< 5,71,71,74
1,female,some college,standard,none,married,sometimes,yes,0.0,school_bus,5 - 10,69,90,88
2,female,master's degree,standard,none,single,sometimes,yes,4.0,school_bus,< 5,87,93,91
3,male,associate's degree,free/reduced,none,married,never,no,1.0,school_bus,5 - 10,45,56,42
4,male,some college,standard,none,married,sometimes,yes,0.0,school_bus,5 - 10,76,78,75


In [17]:
df['lunchtype'].value_counts()

standard        19905
free/reduced    10736
Name: lunchtype, dtype: int64

In [18]:
df.columns = ['is_male', 'parent_educ', 'free_reduced_lunch', 'test_prep_completed', 'parent_marital_status',
       'practicesport', 'is_first_child', 'nrsiblings', 'rides_bus',
       'wkly_study_hours', 'math_score', 'reading_score', 'writing_score']

In [19]:
value_change = {
    'female' : 0,
    'male' : 1,
    'no' : 0,
    'yes': 1,
    'school_bus': 1,
    'private' : 0,
    'none' : 0,
    'completed' : 1,
    'sometimes' : 1,
    'regularly' : 1,
    'never' : 0,
    'free/reduced' : 1,
    'standard' : 0
}

In [20]:
df.replace(to_replace=value_change,inplace=True)

In [21]:
# we are going to engineer the final score.
scores = ['writing_score' ,'reading_score', 'math_score']
df['final_score'] =round( df[scores].mean(axis=1),2)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30641 entries, 0 to 30640
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   is_male                30641 non-null  int64  
 1   parent_educ            30641 non-null  object 
 2   free_reduced_lunch     30641 non-null  int64  
 3   test_prep_completed    30641 non-null  int64  
 4   parent_marital_status  30641 non-null  object 
 5   practicesport          30641 non-null  int64  
 6   is_first_child         30641 non-null  int64  
 7   nrsiblings             30641 non-null  float64
 8   rides_bus              30641 non-null  int64  
 9   wkly_study_hours       30641 non-null  object 
 10  math_score             30641 non-null  int64  
 11  reading_score          30641 non-null  int64  
 12  writing_score          30641 non-null  int64  
 13  final_score            30641 non-null  float64
dtypes: float64(2), int64(9), object(3)
memory usage: 3.3+ 

In [23]:
object_columns = list()
for cols in df.columns:
    if df[cols].dtype == 'O':
        object_columns.append(cols)
    

In [24]:
df = pd.get_dummies(df, columns=object_columns, drop_first= True)

In [25]:
df.columns

Index(['is_male', 'free_reduced_lunch', 'test_prep_completed', 'practicesport',
       'is_first_child', 'nrsiblings', 'rides_bus', 'math_score',
       'reading_score', 'writing_score', 'final_score',
       'parent_educ_bachelor's degree', 'parent_educ_high school',
       'parent_educ_master's degree', 'parent_educ_some college',
       'parent_educ_some high school', 'parent_marital_status_married',
       'parent_marital_status_single', 'parent_marital_status_widowed',
       'wkly_study_hours_< 5', 'wkly_study_hours_> 10'],
      dtype='object')

In [26]:
df.head()

,is_male,free_reduced_lunch,test_prep_completed,practicesport,is_first_child,nrsiblings,rides_bus,math_score,reading_score,writing_score,...,parent_educ_bachelor's degree,parent_educ_high school,parent_educ_master's degree,parent_educ_some college,parent_educ_some high school,parent_marital_status_married,parent_marital_status_single,parent_marital_status_widowed,wkly_study_hours_< 5,wkly_study_hours_> 10
0,0,0,0,1,1,3.0,1,71,71,74,...,1,0,0,0,0,1,0,0,1,0
1,0,0,0,1,1,0.0,1,69,90,88,...,0,0,0,1,0,1,0,0,0,0
2,0,0,0,1,1,4.0,1,87,93,91,...,0,0,1,0,0,0,1,0,1,0
3,1,1,0,0,0,1.0,1,45,56,42,...,0,0,0,0,0,1,0,0,0,0
4,1,0,0,1,1,0.0,1,76,78,75,...,0,0,0,1,0,1,0,0,0,0


In [27]:
def split(df):
    '''
    This function splits a dataframe into 
    train, validate, and test in order to explore the data and to create and validate models. 
    It takes in a dataframe and contains an integer for setting a seed for replication. 
    Test is 20% of the original dataset. The remaining 80% of the dataset is 
    divided between valiidate and train, with validate being .30*.80= 24% of 
    the original dataset, and train being .70*.80= 56% of the original dataset. 
    The function returns, train, validate and test dataframes. 
    '''
    train, test = train_test_split(df, test_size = .2, random_state=123)   
    train, validate = train_test_split(train, test_size=.3, random_state=123)
    
    return train, validate, test

In [28]:
from sklearn.impute import SimpleImputer

def prepare_edu():
    df = a.acquire_edu_data()
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    df.drop(columns=[c for c in df.columns if 'unnamed' in c],inplace=True)
    df.drop(columns='ethnicgroup', inplace=True)
    df.drop(columns=[c for c in df.columns if 'unnamed' in c],inplace=True)
    has_null =[]

    for cols in df.columns:
        if df[cols].isna().sum() > 0:
            has_null.append(cols)
        else:
            pass

    imputer = SimpleImputer(strategy='most_frequent')

    # Fill the missing values in each column
    for col in df.columns:
        if df[col].isna().sum() > 0:
            df[col] = imputer.fit_transform(df[col].values.reshape(-1, 1))


    df.columns = ['is_male', 'parent_educ', 'free_reduced_lunch', 'test_prep_completed', 'parent_marital_status',
           'practicesport', 'is_first_child', 'nrsiblings', 'rides_bus',
           'wkly_study_hours', 'math_score', 'reading_score', 'writing_score']

    value_change = {
    'female' : 0,
    'male' : 1,
    'no' : 0,
    'yes': 1,
    'school_bus': 1,
    'private' : 0,
    'none' : 0,
    'completed' : 1,
    'sometimes' : 1,
    'regularly' : 1,
    'never' : 0,
    'free/reduced' : 1,
    'standard' : 0
    }
    

    df.replace(to_replace=value_change,inplace=True)

    scores = ['writing_score' ,'reading_score', 'math_score']
    df['final_score'] =round( df[scores].mean(axis=1),2)


    object_columns = list()
    for cols in df.columns:
        if df[cols].dtype == 'O':
            object_columns.append(cols)
    df = pd.get_dummies(df, columns=object_columns, drop_first= True)
    df.drop_duplicates(inplace=True)
    return df


In [29]:
df =  prepare_edu()

In [30]:
df.columns

Index(['is_male', 'free_reduced_lunch', 'test_prep_completed', 'practicesport',
       'is_first_child', 'nrsiblings', 'rides_bus', 'math_score',
       'reading_score', 'writing_score', 'final_score',
       'parent_educ_bachelor's degree', 'parent_educ_high school',
       'parent_educ_master's degree', 'parent_educ_some college',
       'parent_educ_some high school', 'parent_marital_status_married',
       'parent_marital_status_single', 'parent_marital_status_widowed',
       'wkly_study_hours_< 5', 'wkly_study_hours_> 10'],
      dtype='object')

In [31]:
df.duplicated().sum()

0